In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img

from skimage.filters import gaussian

import tensorflow as tf
from tensorflow import keras



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/My Drive/ML_Reserch-Paper/data"
test_Data ="/content/drive/My Drive/ML_Reserch-Paper/TestData"
normal_path = os.path.join(data_path, "normal_skin")
lumpy_path = os.path.join(data_path, "lumpy_skin")

Mounted at /content/drive


In [ ]:
# Define target labels
normal_label = 0
lumpy_label = 1

# Define image size
img_width, img_height = 224, 224


In [ ]:
def load_preprocess_image(img_path, label):
    # Load image in grayscale mode
    img = load_img(img_path, color_mode="grayscale", target_size=(img_width, img_height))

    # Convert image to array
    img_array = img_to_array(img)

    # Apply Gaussian blurring
    blurred_img = gaussian(img_array, sigma=1, multichannel=False)

    # Normalize pixel values to range (0, 1)
    blurred_img /= 255.0

    return blurred_img, label


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

images = []
labels = []

for folder in [normal_path, lumpy_path]:
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img_array, label = load_preprocess_image(img_path, normal_label if folder == normal_path else lumpy_label)
        images.append(img_array)
        labels.append(label)

<ipython-input-4-6ceeb60b9de2>:9: FutureWarning: `multichannel` is a deprecated argument name for `gaussian`. It will be removed in version 1.0. Please use `channel_axis` instead.
  blurred_img = gaussian(img_array, sigma=1, multichannel=False)


In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [ ]:
# # Ensure the data is balanced
# min_samples = min(np.sum(labels == normal_label), np.sum(labels == lumpy_label))
# balanced_indices = []
# for label in [normal_label, lumpy_label]:
#     class_indices = np.where(labels == label)[0]
#     balanced_indices.extend(np.random.choice(class_indices, size=min_samples, replace=False))

# images = images[balanced_indices]
# labels = labels[balanced_indices]


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reshape X_train and X_test to the expected format
X_train = X_train.reshape(-1, img_width, img_height, 1)  # Adjust the number of channels if needed (e.g., 1 for grayscale)
X_test = X_test.reshape(-1, img_width, img_height, 1)

# Optional: Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("X_train= ",len(X_train))
print("y_train= ",len(y_train))

print("X_test=  ",len(X_test))
print("y_test = ",len(y_test))


X_train=  259
y_train=  259
X_test=   65
y_test =  65


In [ ]:
# Create training and validation generators (for data augmentation)
train_generator = datagen.flow(X_train, y_train, batch_size=12)
validation_generator = datagen.flow(X_test, y_test, batch_size=12)

# Use these generators for training your machine learning model


In [ ]:
Model using VGG19
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Load the VGG19 model
model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:

# Add new layers to the model
x = model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


In [ ]:
# Create a new model with the added layers
model = Model(inputs=model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator,
          steps_per_epoch=len(X_train) // 32,
          epochs=15,
          validation_data=validation_generator,
          validation_steps=len(X_test) // 32)

In [ ]:
# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test)
print("Overall accuracy:", accuracy)


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Modify the CNN architecture for grayscale images
new_model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),  # Additional dense layer
    Dense(128, activation='relu'),  # Additional dense layer
    Dense(64, activation='relu'),   # Additional dense layer
    Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the model
new_model.compile(optimizer='Adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the model
history = new_model.fit(train_generator,
                        steps_per_epoch=len(X_train) // 32,
                        epochs=25,
                        validation_data=validation_generator,
                        validation_steps=len(X_test) // 32)

Epoch 1/25
8/8 [==============================] - 27s 3s/step - loss: 0.8037 - accuracy: 0.6374 - val_loss: 0.6770 - val_accuracy: 0.5833
Epoch 2/25
8/8 [==============================] - 16s 2s/step - loss: 0.6665 - accuracy: 0.5495 - val_loss: 0.6661 - val_accuracy: 0.6667
Epoch 3/25
8/8 [==============================] - 16s 2s/step - loss: 0.6725 - accuracy: 0.6354 - val_loss: 0.6616 - val_accuracy: 0.6667
Epoch 4/25
8/8 [==============================] - 16s 2s/step - loss: 0.6754 - accuracy: 0.6042 - val_loss: 0.6753 - val_accuracy: 0.6250
Epoch 5/25
8/8 [==============================] - 17s 2s/step - loss: 0.6704 - accuracy: 0.5495 - val_loss: 0.6990 - val_accuracy: 0.5417
Epoch 6/25
8/8 [==============================] - 17s 2s/step - loss: 0.7036 - accuracy: 0.5604 - val_loss: 0.6786 - val_accuracy: 0.6250
Epoch 7/25
8/8 [==============================] - 16s 2s/step - loss: 0.6602 - accuracy: 0.5729 - val_loss: 0.7241 - val_accuracy: 0.5833
Epoch 8/25
8/8 [==================

In [ ]:
# Evaluate the new_model on the testing data
loss, accuracy = new_model.evaluate(validation_generator)
print("Overall accuracy:", accuracy)


6/6 [==============================] - 3s 553ms/step - loss: 0.6119 - accuracy: 0.6923
Overall accuracy: 0.692307710647583


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score
# Get the test data and labels
X_test, y_test = next(iter(validation_generator))

# Make predictions on the test data
y_pred = np.round(new_model.predict(X_test)).flatten()

# Calculate the performance metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("AUC-ROC:", auc_roc)
print("Confusion Matrix:\n", conf_matrix)
print("Matthews Correlation Coefficient:", mcc)
print("Cohen's Kappa:", kappa)

1/1 [==============================] - 1s 599ms/step
Precision: 1.0
Recall: 0.75
F1-Score: 0.8571428571428571
AUC-ROC: 0.875
Confusion Matrix:
 [[8 0]
 [1 3]]
Matthews Correlation Coefficient: 0.816496580927726
Cohen's Kappa: 0.8


In [ ]:
# from google.colab import files

# files.upload()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the single image
img_path = "augmented_image_2.jpg"  # Replace with the path to your single image
img = image.load_img(img_path, target_size=(img_width, img_height), color_mode="grayscale")

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand the dimensions of the image array to match the input shape expected by the model
# (assuming your model expects input shape of (224, 224, 1))
img_array = np.expand_dims(img_array, axis=0)

# Normalize the pixel values (if needed)
# img_array = img_array / 255.0  # Assuming pixel values are in the range [0, 255]

# Make predictions using the model
predictions = new_model.predict(img_array)

# Interpret the predictions
if predictions[0] > 0.5:
    print("The image is classified as lumpy skin.")
else:
    print("The image is classified as normal skin.")


1/1 [==============================] - 0s 284ms/step
The image is classified as lumpy skin.


In [ ]:

# import numpy as np
# from tensorflow.keras.preprocessing.image import array_to_img

# # Define a directory to store the augmented images
# augmented_data_dir = "/content/drive/My Drive/ML_Reserch-Paper/AugmentedData"
# os.makedirs(augmented_data_dir, exist_ok=True)

# # Define a function to save augmented images
# def save_augmented_images(generator, num_images):
#     # Reset the generator to start at the beginning
#     generator.reset()

#     # Loop through the generator to generate and save augmented images
#     for i in range(num_images):
#         # Generate a batch of augmented images
#         batch_images, _ = next(generator)

#         # Loop through the batch and save each augmented image
#         for j in range(len(batch_images)):
#             # Convert the augmented image from numpy array to PIL Image
#             augmented_img = array_to_img(batch_images[j])

#             # Save the augmented image to the specified directory
#             img_path = os.path.join(augmented_data_dir, f"augmented_image_{i * len(batch_images) + j}.jpg")
#             augmented_img.save(img_path)

# # Save augmented images from the training generator
# save_augmented_images(train_generator, num_images=100)


In [ ]:
import os
from tensorflow.keras.models import save_model
from tensorflow.keras.optimizers import Adamax
# # Define the path to save the model
model_save_path = os.path.join("/content/drive/My Drive", "ML_Reserch-Paper", "modelAdam.h5")

# # Save the model
new_model.save(model_save_path)

# # new_model.save('my_model.h5', save_format='h5', include_optimizer=False,
# #                 signatures=None, options=None, save_traces=True,
# #                 **kwargs)
# model_save_path = os.path.join("/content/drive/My Drive", "ML_Reserch-Paper", "modelCNN.h5")
# new_model.save(model_save_path, custom_objects={'Adamax': Adamax})
# new_model.save('my_model.h5', save_format='h5', include_optimizer=False,
#                  signatures=None, options=None, save_traces=True,
#                  **kwargs)
# import pickle
# import os

# # Save the model to a .pkl file
# model_save_path = os.path.join("/content/drive/My Drive", "ML_Reserch-Paper", "my_model.pkl")
# with open(model_save_path, 'wb') as file:
#     pickle.dump(new_model, file)

NameError: name 'kwargs' is not defined

In [ ]:
from flask import Flask, request, jsonify
import base64
import io
import tensorflow as tf

app = Flask(__name__)
new_model = ...  # Load your machine learning model here

@app.route('/classify-image', methods=['POST'])
def classify_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    img_file = request.files['image']
    if img_file.filename == '':
        return jsonify({'error': 'No image provided'}), 400

    img = tf.io.decode_image(img_file.read(), channels=1)
    img = tf.image.resize(img, [img_width, img_height])
    img_array = np.expand_dims(img, axis=0)
    predictions = new_model.predict(img_array)

    if predictions[0] > 0.5:
        result = "The image is classified as lumpy skin."
    else:
        result = "The image is classified as normal skin."

    return jsonify({'result': result})

if __name__ == '__main__':
    app.run(debug=True)